In [40]:
import re
import pyodbc
import pandas as pd
from pandas import DataFrame
from geotext import GeoText #pip install geotext
import numpy as np

In [2]:

conn = pyodbc.connect('DSN=SQLAMLIN;UID=MSAmlin;PWD=password')
cur = conn.cursor()
table = '[initial].[LondonFireRecent]'
sql = 'select * from {}'.format(table)
df = pd.read_sql(sql, conn)

d = df[['Text']].to_dict()

locdic = {} # initialises locdic dictionary
succ = {}
succd = {}

roadarray = ['ROAD', 'LANE', 'DRIVE', 'WAY', 'STREET', 'AVENUE']
# array of locational keywords

#interates through the numbered keys of text dictionary
for i in d['Text']:
    text = d['Text'][i] # extracts text for each iteration
    address = {} # initialises address dictionary
    for word in roadarray: # iterates through each word in keywords
        try:
            search  = r'((?:\w+(?:\s+|$)){1})' + word # search for the word before the key word iterated
            road = re.search(search, text, re.IGNORECASE) #  assigns var raod to the search query ignoring case
            address['ROAD'] = road.group() # address['ROAD'] is the word before keyword+keyword
        except:
            pass
        
    # If address['ROAD'] still doesn't exist, assign it as UNKNOWN   
    try:
        address['ROAD']
    except:
        address['ROAD'] = 'UNKNOWN'
    
    # Assigns TOWN as the geotext identified city
    try:
        address['TOWN'] = GeoText(text).cities[-1] # The last entry of cities is the most granular
    except:   
        address['TOWN'] = 'UNKNOWN' # Sets it to null value if town didnt match anything
    
    locdic[i] = address # appends the address dictionary into the final locdic
    
locdf = pd.DataFrame.from_dict(locdic, orient='index') # transforms locdic into a df

# Performance metrics
missrows = locdf[(locdf['ROAD']=='UNKNOWN') & (locdf['TOWN']== 'UNKNOWN')].shape[0]
missroad = locdf.loc[locdf['ROAD'] == 'UNKNOWN'].shape[0]
misstown = locdf.loc[locdf['TOWN'] == 'UNKNOWN'].shape[0]
totalrows = locdf.shape[0]
TotSuc = 'Success Rate Total = ' + str(round((((totalrows - missrows)/totalrows)*100), 2))
RdSuc = 'Success Rate ROAD = ' + str(round((((totalrows - missroad)/totalrows)*100), 2))
TnSuc = 'Success Rate TOWN = ' + str(round((((totalrows - misstown)/totalrows)*100), 2))
print(TotSuc)
print(RdSuc)
print(TnSuc)
succ['TotalSuccess'] = TotSuc
succ['RoadSuccess'] = RdSuc
succ['TownSuccess'] = TnSuc
succd[table] = succ

Success Rate Total = 90.18
Success Rate ROAD = 72.07
Success Rate TOWN = 67.33


In [7]:
text = df.iloc[0]['Text']
text

'Over 97 firefighters and officers tackled a fire in a pub on Bellegrove Road, Welling this morning.\r\nSevenÂ\xa0people left the building before the Brigade arrived and there were no reports of any injuries.\r\nAround half of the first floor andÂ\xa0most of the roof was damaged in the fire.\r\nStation Manager Martin Freeman who was at the scene said;\r\n"Staff at the pub had a lucky escape as they heard the smoke alarm sound, left the pub and alerted the Brigade. I canâ€™t stress how important smoke alarms are, you should have one on every floor of your property and test them regularly.\r\n"Thanks to the alarm, seven members of staff were able to leave the building in good time but had they not had an early warning there could have been a very different outcome.â€\x9d\r\nFifteenÂ\xa0fire engines andÂ\xa0two aerial ladder platforms were at the scene from stations including East Greenwich, Plumstead, Erith, Lee Green, Eltham, Deptford and Bexley.\r\nThe Brigade was called at 0924 and th

In [49]:
def anlun_address(text):
    from geotext import GeoText #pip install geotext

    roadarray = ['ROAD', 'LANE', 'DRIVE', 'WAY', 'STREET', 'AVENUE']
    # array of locational keywords

    #interates through the numbered keys of text dictionary
    
    address = {} # initialises address dictionary
    for word in roadarray: # iterates through each word in keywords
        try:
            search  = r'((?:\w+(?:\s+|$)){1})' + word # search for the word before the key word iterated
            road = re.search(search, text, re.IGNORECASE) #  assigns var raod to the search query ignoring case
            address['ROAD'] = road.group() # address['ROAD'] is the word before keyword+keyword
        except:
            pass

    # If address['ROAD'] still doesn't exist, assign it as UNKNOWN   
    try:
        address['ROAD']
    except:
        address['ROAD'] = 'UNKNOWN'

    # Assigns TOWN as the geotext identified city
    try:
        address['TOWN'] = GeoText(text).cities[-1] # The last entry of cities is the most granular
    except:   
        address['TOWN'] = 'UNKNOWN' # Sets it to null value if town didnt match anything
    
    address # appends the address dictionary into the final locdic
    #return address['ROAD'] + ' ' + address['TOWN']
    #locdf = pd.DataFrame(address)#.from_dict(address, orient='index') # transforms locdic into a df
    address_string = address['ROAD'] + ' '+ address['TOWN']
    return address_string

In [50]:
anlun_address(text)

'Bellegrove Road Bexley'

In [45]:
df['RoadAddress'] = df['Text'].map(lambda x: anlun_address(x)['ROAD'])
df['TownAddress'] = df['Text'].map(lambda x: anlun_address(x)['TOWN'])
df['TownAddress2'] = df['TownAddress'].replace('UNKNOWN', np.nan)
df['TownAddress2'] = df['TownAddress2'].fillna(df['Location'])

In [46]:
df.head(30)

,IncidentID,Caption,Date,Location,Text,Time,Address,RoadAddress,TownAddress,TownAddress2
0,1,100 firefighters tackled Welling pub fire,31/08/2015,Westminster,Over 97 firefighters and officers tackled a fi...,31/08/2015,Bellegrove Road Bexley,Bellegrove Road,Bexley,Bexley
1,2,12 fire engines tackle Southwark apartment blo...,07/02/2017,Southwark,Twelve fire engines and 81 firefighters and of...,07/02/2017,Guildford Street Brixton,Guildford Street,Brixton,Brixton
2,3,12 people evacuate flat fire in Bermondsey,25/11/2015,Southwark,Twelve people escaped uninjured from a fire at...,25/11/2015,Grange Road UNKNOWN,Grange Road,UNKNOWN,Southwark
3,4,120 firefighters tackle Tottenham warehouse blaze,06/11/2015,Tower Hamlets,Twenty fire engines and aroundÂ 120 firefighte...,06/11/2015,Pretoria Road Edmonton,Pretoria Road,Edmonton,Edmonton
4,5,20 fire engines and 120 firefighters attended ...,06/06/2018,Westminster,TwentyÂ fire engines and 120 firefighters and ...,06/06/2018,UNKNOWN Battersea,UNKNOWN,Battersea,Battersea
5,6,20 firefighters tackled a house fire in Romford,29/05/2018,Havering,Four fire engines and 21 firefighters and offi...,29/05/2018,Dagenham Road Hornchurch,Dagenham Road,Hornchurch,Hornchurch
6,7,200 people evacuate from fire at Southwark hal...,15/11/2015,Southwark,Two hundred people safely evacuated from a stu...,15/11/2015,Stamford Street UNKNOWN,Stamford Street,UNKNOWN,Southwark
7,8,21 firefighters and officers fight fast food fire,17/02/2018,Waltham Forest,Four fire engines and 21 firefighters and offi...,17/02/2018,The Avenue UNKNOWN,The Avenue,UNKNOWN,Waltham Forest
8,9,21 firefighters attended the scene,05/04/2018,Kingston upon Thames,Four fire engines and 21 firefighters and offi...,05/04/2018,Galsworthy Road New Malden,Galsworthy Road,New Malden,New Malden
9,10,21 firefighters attended the scene,05/04/2018,Lambeth,Four fire engines and 21 firefighters and offi...,05/04/2018,Larkhall Lane Brixton,Larkhall Lane,Brixton,Brixton


In [21]:
df['Address'] = df['Text'].apply(anlun_address)

In [22]:
df['Address'] = df['Address'].map(lambda x: x[:-7]+ if (x.split()[-1] == 'UNKNOWN'))

,IncidentID,Caption,Date,Location,Text,Time,Address
0,1,100 firefighters tackled Welling pub fire,31/08/2015,Westminster,Over 97 firefighters and officers tackled a fi...,31/08/2015,Bellegrove Road Bexley
1,2,12 fire engines tackle Southwark apartment blo...,07/02/2017,Southwark,Twelve fire engines and 81 firefighters and of...,07/02/2017,Guildford Street Brixton
2,3,12 people evacuate flat fire in Bermondsey,25/11/2015,Southwark,Twelve people escaped uninjured from a fire at...,25/11/2015,Grange Road UNKNOWN
3,4,120 firefighters tackle Tottenham warehouse blaze,06/11/2015,Tower Hamlets,Twenty fire engines and aroundÂ 120 firefighte...,06/11/2015,Pretoria Road Edmonton
4,5,20 fire engines and 120 firefighters attended ...,06/06/2018,Westminster,TwentyÂ fire engines and 120 firefighters and ...,06/06/2018,UNKNOWN Battersea
5,6,20 firefighters tackled a house fire in Romford,29/05/2018,Havering,Four fire engines and 21 firefighters and offi...,29/05/2018,Dagenham Road Hornchurch
6,7,200 people evacuate from fire at Southwark hal...,15/11/2015,Southwark,Two hundred people safely evacuated from a stu...,15/11/2015,Stamford Street UNKNOWN
7,8,21 firefighters and officers fight fast food fire,17/02/2018,Waltham Forest,Four fire engines and 21 firefighters and offi...,17/02/2018,The Avenue UNKNOWN
8,9,21 firefighters attended the scene,05/04/2018,Kingston upon Thames,Four fire engines and 21 firefighters and offi...,05/04/2018,Galsworthy Road New Malden
9,10,21 firefighters attended the scene,05/04/2018,Lambeth,Four fire engines and 21 firefighters and offi...,05/04/2018,Larkhall Lane Brixton
